In [1]:
import pandas as pd
import numpy as np
import random

# Paso 2: Modelado de Monte Carlo
def simular_partido(partidos, equipo_local, equipo_visitante):
    # Filtra el historial para obtener los partidos entre los dos equipos
    partidos_entre_equipos = partidos[
        ((partidos['Home'] == equipo_local) & (partidos['Away'] == equipo_visitante)) |
        ((partidos['Home'] == equipo_visitante) & (partidos['Away'] == equipo_local))
    ]
    
    # Si no hay historial, simplemente asignamos una probabilidad del 50% de ganar a cada equipo
    if len(partidos_entre_equipos) == 0:
        return random.choice(['local', 'empate', 'visitante'])
    
    # Calculamos la probabilidad de victoria basándonos en el historial de partidos
    victorias_local = sum(partidos_entre_equipos['Score (Home)'] > partidos_entre_equipos['Score (Away)'])
    victorias_visitante = sum(partidos_entre_equipos['Score (Home)'] < partidos_entre_equipos['Score (Away)'])
    probabilidad_local = victorias_local / len(partidos_entre_equipos)
    probabilidad_visitante = victorias_visitante / len(partidos_entre_equipos)
    
    # Simulamos el resultado del partido basándonos en estas probabilidades
    resultado = random.choices(['local', 'empate', 'visitante'], weights=[probabilidad_local, 1 - probabilidad_local - probabilidad_visitante, probabilidad_visitante])[0]
    return resultado

def simular_partidos_monte_carlo(partidos, equipos_locales, equipos_visitantes, num_simulaciones):
    resultados = []
    for i in range(num_simulaciones):
        resultados_simulados = []
        for local, visitante in zip(equipos_locales, equipos_visitantes):
            resultado_simulado = simular_partido(partidos, local, visitante)
            resultados_simulados.append(resultado_simulado)
        resultados.append(resultados_simulados)
    return resultados

# Paso 3: Determinación de ganadores
def calcular_probabilidades(resultados_simulados, equipo):
    # Calcula la probabilidad de victoria del equipo en base a los resultados simulados
    total_simulaciones = len(resultados_simulados)
    victorias_equipo = sum(1 for resultado in resultados_simulados if resultado == equipo)
    return victorias_equipo / total_simulaciones

# Paso 4: Predicción de finalistas
def predecir_finalistas(semis, probabilidades_equipo_local, probabilidades_equipo_visitante):
    # Basándote en las probabilidades de victoria, determina qué equipos tienen más probabilidades de avanzar a la final
    # En este caso, seleccionaremos el equipo con la mayor probabilidad de victoria en cada partido
    finalistas = []
    for prob_local, prob_visitante in zip(probabilidades_equipo_local, probabilidades_equipo_visitante):
        if prob_local > prob_visitante:
            finalistas.append('local')
        else:
            finalistas.append('visitante')
    return finalistas

# Ejemplo de uso
partidos = pd.read_csv('../../data/partidos_limpio.csv')

# Definimos una lista llamada 'semis' que contiene datos de partidos de fútbol de las semifinales
semis = [['2023-2024', 'Semi-finals', 'Tue', 'Bayern Munich', 'Germany', 107.882298136646, 114.5545351473923, 'Spain', 'Real Madrid', 'Allianz Arena', 23, 28.3, 10, 110, 18, 14, 32, 16, 2, 2, 13.0, 1.0, 22, 28.0, 10, 110, 20, 17, 37, 20, 0, 1, 18.0, 0.0, 2024, 4, 30],
        ['2023-2024', 'Semi-finals', 'Wed', 'Dortmund', 'Germany', 91.17303312629399, 114.33458049886625, 'France', 'Paris S-G', 'Signal Iduna Park', 23, 28.0, 10, 110, 15, 12, 27, 14, 1, 1, 16.0, 0.0, 21, 25.3, 10, 110, 19, 12, 31, 16, 3, 3, 27.0, 0.0, 2024, 5, 1],
        ['2023-2024', 'Semi-finals', 'Tue', 'Paris S-G', 'France', 114.33458049886625, 91.17303312629399, 'Germany', 'Dortmund', 'Parc des Princes', 21, 25.3, 10, 110, 19, 12, 31, 16, 3, 3, 27.0, 0.0, 23, 28.0, 10, 110, 15, 12, 27, 14, 1, 1, 16.0, 0.0, 2024, 5, 7],
        ['2023-2024', 'Semi-finals', 'Wed', 'Real Madrid', 'Spain', 114.5545351473923, 107.882298136646, 'Germany', 'Bayern Munich', 'Estadio Santiago Bernabéu', 22, 28.0, 10, 110, 20, 17, 37, 20, 0, 1, 18.0, 0.0, 23, 28.3, 10, 110, 18, 14, 32, 16, 2, 2, 13.0, 1.0, 2024, 5, 8]]

# Obtenemos las columnas relevantes del DataFrame 'partidos' para usar como nombres de columnas en el DataFrame 'semis'
partidos_cols = partidos.drop(labels=['Results', 'Score (Home)', 'Score (Away)', 'Referee'], axis=1).columns

# Creamos un DataFrame 'semis' a partir de la lista 'semis' con las columnas obtenidas del DataFrame 'partidos'
semis = pd.DataFrame(semis, columns=(partidos_cols))

num_simulaciones = 1000
resultados_simulados = simular_partidos_monte_carlo(partidos, semis['Home'], semis['Away'], num_simulaciones)

# Calcula las probabilidades de victoria para cada equipo en los partidos futuros
probabilidades_equipo_local = [calcular_probabilidades(resultados, 'local') for resultados in resultados_simulados]
probabilidades_equipo_visitante = [1 - prob for prob in probabilidades_equipo_local]

# Predice los finalistas
finalistas = predecir_finalistas(semis, probabilidades_equipo_local, probabilidades_equipo_visitante)
print("Finalistas predichos:", finalistas)



ValueError: 35 columns passed, passed data had 37 columns